In [ ]:
# PocketMKT: Guia Prático para Rodar LLM LoRA/PEFT na AWS GPU ou Nuvem

## 1. Configuração Inicial

```bash
# (RODE NO TERMINAL LOCAL)
# Conectar por SSH
ssh -i pocketmkt-gpu.pem ubuntu@SEU_IP_PUBLICO_AQUI

# Subir arquivo do modelo já treinado (.zip)
scp -i pocketmkt-gpu.pem /caminho/local/fine_tuned_advocacia.zip ubuntu@SEU_IP_PUBLICO_AQUI:/home/ubuntu/
```

---

In [ ]:
## 2. Instale as Bibliotecas Necessárias

```bash
pip install torch transformers peft accelerate huggingface_hub flask
```

---

In [ ]:
## 3. (Opcional) Ative o Ambiente PyTorch

```bash
source /opt/pytorch/bin/activate
```

---

In [ ]:
## 4. Configure Cache/Offload para Modelos Grandes (se necessário)

```bash
mkdir -p /opt/dlami/nvme/hf_cache
mkdir -p /opt/dlami/nvme/offload
chmod 777 /opt/dlami/nvme/hf_cache /opt/dlami/nvme/offload
export HF_HOME=/opt/dlami/nvme/hf_cache
export TRANSFORMERS_CACHE=/opt/dlami/nvme/hf_cache
```

---

In [ ]:
## 5. Login Hugging Face (para modelos gated)

```bash
~/.local/bin/huggingface-cli login
# ou, se preferir:
pip install hfcli
hf auth login
```

---

In [ ]:
## 6. Descompacte o Modelo

```bash
unzip fine_tuned_advocacia.zip
```

---


In [ ]:
## 7. Teste o Carregamento do Modelo LoRA/PEFT

```python
import os
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

os.environ["HF_HOME"] = "/opt/dlami/nvme/hf_cache"
os.environ["TRANSFORMERS_CACHE"] = "/opt/dlami/nvme/hf_cache"

adapter_path = "content/fine_tuned_advocacia"
peft_config = PeftConfig.from_pretrained(adapter_path)
base_model = peft_config.base_model_name_or_path

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    torch_dtype=torch.float16,
    device_map="auto",
    offload_folder="/opt/dlami/nvme/offload"
)
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = PeftModel.from_pretrained(
    model,
    adapter_path,
    offload_folder="/opt/dlami/nvme/offload"
)

prompt = "Olá, advogado! Como posso te ajudar?"
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=64)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))
```

---

In [ ]:
## 8. Suba um Servidor Flask para API HTTP

```python
from flask import Flask, request, jsonify
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch, os

os.environ["HF_HOME"] = "/opt/dlami/nvme/hf_cache"
os.environ["TRANSFORMERS_CACHE"] = "/opt/dlami/nvme/hf_cache"

adapter_path = "content/fine_tuned_advocacia"
peft_config = PeftConfig.from_pretrained(adapter_path)
base_model = peft_config.base_model_name_or_path

model = AutoModelForCausalLM.from_pretrained(
    base_model, torch_dtype=torch.float16, device_map="auto", offload_folder="/opt/dlami/nvme/offload"
)
tokenizer = AutoTokenizer.from_pretrained(base_model)
model = PeftModel.from_pretrained(
    model, adapter_path, offload_folder="/opt/dlami/nvme/offload"
)

app = Flask(__name__)

@app.route('/infer', methods=['POST'])
def infer():
    prompt = request.json.get("prompt")
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=128)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return jsonify({"response": response})

if __name__ == '__main__':
    app.run(host="0.0.0.0", port=8000)
```

---

In [ ]:
## 9. Libere a porta 8000 no Security Group da AWS
- No painel EC2 > Security Groups > Adicione regra para porta 8000 (seu IP)

---

In [ ]:
## 10. Faça request da sua máquina local

```python
import requests
url = "http://SEU_IP_AWS:8000/infer"
data = {"prompt": "Olá, advogado! Como posso te ajudar?"}
resp = requests.post(url, json=data)
print(resp.json())
```

---

In [ ]:
## 11. Dicas Rápidas
- Monitore uso com `df -h`, `top`, `nvidia-smi`
- Limpe caches antigos com `rm -rf /opt/dlami/nvme/hf_cache/*`
- Sempre desligue a instância AWS para não pagar desnecessário!

---

In [ ]:
## 12. Problemas comuns
- Erro "No space left on device": Libere espaço e use disco temporário se possível
- Erro de login Hugging Face: Relogue e aceite os termos do modelo
- Erro de permissão: use `chmod 777` nos diretórios temporários para prototipagem